# Загрузка и предобработка данных

В этом ноутбуке выполняется загрузка исторических данных OHLCV, вычисление log_returns и подготовка базового датафрейма.  

**Ключевые задачи:**
- Загрузка для всех нужных тикеров MOEX
- Чистка пропусков и сплит-дат
- Вычисление log_returns без корректировки по дивидендам/корп. действиям (по решению)
- Сохранение промежуточного результата для Feature Engineering


In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


✅ Библиотеки загружены


In [9]:
# Определение пути к данным
current_dir = Path.cwd()
DATA_DIR = current_dir / "data" / "MOEX_DATA"

# Поиск данных в альтернативных местах
if not DATA_DIR.exists():
    alt_path = current_dir.parent / "ML" / "data" / "MOEX_DATA"
    if alt_path.exists():
        DATA_DIR = alt_path
    else:
        for parent in [current_dir, current_dir.parent]:
            test_path = parent / "data" / "MOEX_DATA"
            if test_path.exists():
                DATA_DIR = test_path
                break

print(f"Путь к данным: {DATA_DIR}")
print(f"Найдено: {DATA_DIR.exists()}")


Путь к данным: /Users/nikitabaslykov/Documents/Python/Trading/MOEXScanner/ML/data/MOEX_DATA
Найдено: True


## Функции загрузки и расчета


In [ ]:
def calculate_log_returns(prices):
    """Вычисляет логарифмические доходности: log(P_t / P_{t-1})"""
    if not isinstance(prices, pd.Series):
        prices = pd.Series(prices)
    log_prices = np.log(prices)
    return log_prices - log_prices.shift(1)


def load_ticker_data(ticker, data_dir=DATA_DIR, timeframe='1D'):
    """Загружает данные по тикеру из CSV"""
    ticker_dir = data_dir / ticker / timeframe
    if not ticker_dir.exists():
        raise FileNotFoundError(f"Папка {ticker_dir} не найдена")
    
    csv_files = list(ticker_dir.glob("*.csv"))
    if not csv_files:
        raise FileNotFoundError(f"CSV файл не найден в {ticker_dir}")
    
    df = pd.read_csv(csv_files[0])
    
    # Преобразование дат
    if 'end' in df.columns:
        df['date'] = pd.to_datetime(df['end'])
    elif 'begin' in df.columns:
        df['date'] = pd.to_datetime(df['begin'])
    elif 'trade_session_date' in df.columns:
        # Для индекса IMOEX
        df['date'] = pd.to_datetime(df['trade_session_date'])
    
    df = df.sort_values('date').reset_index(drop=True)
    
    # Нормализуем: оставляем только необходимые колонки
    required_cols = ['date', 'close']
    available_cols = [col for col in required_cols if col in df.columns]
    df = df[available_cols]
    
    return df

print("✅ Функции загружены")


✅ Функции загружены


## Тестирование на одном тикере


In [11]:
# Тестирование на одном тикере
ticker = 'SBER'
df = load_ticker_data(ticker, timeframe='1D')
df['log_return'] = calculate_log_returns(df['close'])

print(f"\nТикер: {ticker}")
print(f"Период: {df['date'].min().date()} → {df['date'].max().date()}")
print(f"Записей: {len(df)}")
print(f"\nСтатистика log returns:")
print(df['log_return'].describe())



Тикер: SBER
Период: 2020-10-12 → 2025-10-11
Записей: 1302

Статистика log returns:
count    1301.000000
mean        0.000247
std         0.023760
min        -0.455918
25%        -0.007616
50%         0.000539
75%         0.009146
max         0.117686
Name: log_return, dtype: float64


## Загрузка нескольких тикеров


In [ ]:
# Загрузка нескольких тикеров
test_tickers = ['AFKS', 'AFLT', 'ALRS', 'BELU', 'BSPB', 'CHMF', 'FIVE', 
    'GAZP', 'GMKN', 'HYDR', 'IRAO', 'LENT', 'LKOH', 'MAGN', 
    'MGNT', 'MTSS', 'NLMK', 'NVTK', 'OZON', 'PIKK', 'PLZL', 
    'ROSN', 'RTKM', 'SBER', 'SNGS', 'TATN', 'TCSG', 'VKCO', 
    'VTBR', 'YNDX']
results = {}

for ticker in test_tickers:
    try:
        df = load_ticker_data(ticker, timeframe='1D')
        df['log_return'] = calculate_log_returns(df['close'])
        
        # Чистка пропусков
        df = df.dropna(subset=['close', 'log_return'])
        
        results[ticker] = df
        print(f"✅ {ticker}: {len(df)} записей")
    except Exception as e:
        print(f"❌ {ticker}: {e}")

# Загрузка индекса IMOEX отдельно
try:
    index_df = load_ticker_data('IMOEX', timeframe='1D')
    index_df['log_return'] = calculate_log_returns(index_df['close'])
    index_df = index_df.dropna(subset=['close', 'log_return'])
    results['IMOEX'] = index_df
    print(f"✅ IMOEX (индекс): {len(index_df)} записей")
except Exception as e:
    print(f"❌ IMOEX: {e}")

# Сводная таблица
if results:
    summary = pd.DataFrame({
        'Тикер': list(results.keys()),
        'Записей': [len(results[k]) for k in results.keys()],
        'Среднее': [results[k]['log_return'].mean() for k in results.keys()],
        'Ст.откл.': [results[k]['log_return'].std() for k in results.keys()],
    })
    print(f"\n{summary.to_string(index=False)}")


✅ AFKS: 1301 записей
✅ AFLT: 1301 записей
✅ ALRS: 1301 записей
✅ BELU: 1289 записей
✅ BSPB: 1295 записей
✅ CHMF: 1301 записей
✅ FIVE: 242 записей
✅ GAZP: 1301 записей
✅ GMKN: 1297 записей
✅ HYDR: 1301 записей
✅ IRAO: 1299 записей
✅ LENT: 990 записей
✅ LKOH: 1301 записей
✅ MAGN: 1301 записей
✅ MGNT: 1301 записей
✅ MTSS: 1300 записей
✅ NLMK: 1301 записей
✅ NVTK: 1297 записей
✅ OZON: 1205 записей
✅ PIKK: 1301 записей
✅ PLZL: 1297 записей
✅ ROSN: 1301 записей
✅ RTKM: 1301 записей
✅ SBER: 1301 записей
✅ SNGS: 1301 записей
✅ TATN: 1299 записей
✅ TCSG: 1009 записей
✅ VKCO: 976 записей
✅ VTBR: 1297 записей
✅ YNDX: 356 записей

Тикер  Записей   Среднее  Ст.откл.
 AFKS     1301 -0.000517  0.025570
 AFLT     1301 -0.000116  0.025475
 ALRS     1301 -0.000510  0.021944
 BELU     1289 -0.000918  0.063728
 BSPB     1295  0.001606  0.027076
 CHMF     1301 -0.000083  0.023466
 FIVE      242 -0.000585  0.018532
 GAZP     1301 -0.000293  0.026512
 GMKN     1297 -0.000337  0.019813
 HYDR     1301 -0.00050

## Сохранение результатов

Сохраняем промежуточные данные для следующих этапов Feature Engineering


In [13]:
# Сохранение промежуточного результата
OUTPUT_DIR = Path('data') / 'processed'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

for ticker, df in results.items():
    output_path = OUTPUT_DIR / f"{ticker}_ohlcv_returns.parquet"
    df.to_parquet(output_path, index=False)
    print(f"✅ Сохранено: {output_path}")

print(f"\n📁 Все данные сохранены в {OUTPUT_DIR}")


✅ Сохранено: data/processed/AFKS_ohlcv_returns.parquet
✅ Сохранено: data/processed/AFLT_ohlcv_returns.parquet
✅ Сохранено: data/processed/ALRS_ohlcv_returns.parquet
✅ Сохранено: data/processed/BELU_ohlcv_returns.parquet
✅ Сохранено: data/processed/BSPB_ohlcv_returns.parquet
✅ Сохранено: data/processed/CHMF_ohlcv_returns.parquet
✅ Сохранено: data/processed/FIVE_ohlcv_returns.parquet
✅ Сохранено: data/processed/GAZP_ohlcv_returns.parquet
✅ Сохранено: data/processed/GMKN_ohlcv_returns.parquet
✅ Сохранено: data/processed/HYDR_ohlcv_returns.parquet
✅ Сохранено: data/processed/IRAO_ohlcv_returns.parquet
✅ Сохранено: data/processed/LENT_ohlcv_returns.parquet
✅ Сохранено: data/processed/LKOH_ohlcv_returns.parquet
✅ Сохранено: data/processed/MAGN_ohlcv_returns.parquet
✅ Сохранено: data/processed/MGNT_ohlcv_returns.parquet
✅ Сохранено: data/processed/MTSS_ohlcv_returns.parquet
✅ Сохранено: data/processed/NLMK_ohlcv_returns.parquet
✅ Сохранено: data/processed/NVTK_ohlcv_returns.parquet
✅ Сохранен